In [2]:
import astropy.time
import sp3
import typing
import pathlib

# Ensure `sp3` is installed: python3 -m pip install sp3

def download_and_interpolate_sp3(sp3_id: str, start_date: str, end_date: str, download_dir: typing.Union[str, bytes, pathlib.Path]):
    # Define the SP3 ID and the time range
    satellite_id = sp3.NoradId(sp3_id)
    
    # Generate time range spaced one minute apart
    start = astropy.time.Time(start_date)
    end = astropy.time.Time(end_date)
    delta_minutes = astropy.time.TimeDelta(60, format='sec')
    obstime = start + delta_minutes * range(int((end - start).jd * 24 * 60))  # Convert delta to minutes

    # Download and interpolate SP3 data to get ITRS positions
    itrs_positions = sp3.itrs(
        id=satellite_id,
        obstime=obstime,
        download_directory=download_dir,
        window=5,  # Default window size for interpolation
        degree=10  # Default polynomial degree for interpolation
    )

    # Extract and print X, Y, Z coordinates in ITRS (ECEF) frame for each minute
    for pos in itrs_positions:
        print(f"Time: {pos.obstime.utc.iso}, X: {pos.x.value} m, Y: {pos.y.value} m, Z: {pos.z.value} m")

# Example usage
download_and_interpolate_sp3("41328", "2023-01-01T00:00:00", "2023-01-02T00:00:00", "sp3_download")


Time: 2023-01-01 00:00:00.000, X: 13487499.125195889 m, Y: 20532040.918172546 m, Z: 10510449.061968252 m
Time: 2023-01-01 00:01:00.000, X: 13417451.52417864 m, Y: 20493075.536434643 m, Z: 10676136.253331443 m
Time: 2023-01-01 00:02:00.000, X: 13346308.236231413 m, Y: 20453578.746938188 m, Z: 10841020.25754173 m
Time: 2023-01-01 00:03:00.000, X: 13274068.710757729 m, Y: 20413562.444138333 m, Z: 11005088.717814462 m
Time: 2023-01-01 00:04:00.000, X: 13200732.561543293 m, Y: 20373038.552816603 m, Z: 11168329.336588668 m
Time: 2023-01-01 00:05:00.000, X: 13126299.565691153 m, Y: 20332019.026962128 m, Z: 11330729.881287113 m
Time: 2023-01-01 00:06:00.000, X: 13050769.664751088 m, Y: 20290515.84703169 m, Z: 11492278.182017459 m
Time: 2023-01-01 00:07:00.000, X: 12974142.964740537 m, Y: 20248541.017995358 m, Z: 11652962.133196961 m
Time: 2023-01-01 00:08:00.000, X: 12896419.736485478 m, Y: 20206106.56714683 m, Z: 11812769.69403873 m
Time: 2023-01-01 00:09:00.000, X: 12817600.415192138 m, Y: 2

In [5]:
import sp3
import astropy.time
import pathlib

def download_sp3_file(sp3_id: str, start_date: str, end_date: str, download_dir: pathlib.Path):
    # Ensure the download directory exists
    download_dir.mkdir(parents=True, exist_ok=True)

    # Define the SP3 ID and the time range
    satellite_id = sp3.NoradId(sp3_id)
    start = astropy.time.Time(start_date)
    end = astropy.time.Time(end_date)
    
    # Calculate the range to download SP3 files without interpolation
    delta_days = astropy.time.TimeDelta(1, format='jd')  # 1 day step to ensure all files are covered
    obstime = start + delta_days * range(int((end - start).jd) + 1)  # Cover entire range day by day

    # Download SP3 files
    for single_day in obstime:
        try:
            sp3.itrs(
                id=satellite_id,
                obstime=astropy.time.Time([single_day.iso]),
                download_directory=download_dir,
                window=1,  # Minimal window since we're not interpolating
                degree=1  # Minimal degree since we're not interpolating
            )
            print(f"Downloaded SP3 file for {single_day.iso} in {download_dir}")
        except Exception as e:
            print(f"Failed to download or find SP3 file for {single_day.iso}: {str(e)}")

# Example usage: Adjust the NORAD ID and time range as needed
download_sp3_file("25933", "2023-01-01T00:00:00", "2023-01-02T00:00:00", pathlib.Path("sp3_download"))


Downloaded SP3 file for 2023-01-01 00:00:00.000 in sp3_download
Downloaded SP3 file for 2023-01-02 00:00:00.000 in sp3_download
